# Solving nonlinear Equations in Julia

There are four types of nonlinear systems:

 1. The "standard nonlinear system", i.e. the `NonlinearProblem`. This is a system of
    equations with an initial condition where you want to satisfy all equations
    simultaneously.
 2. The "interval rootfinding problem", i.e. the `IntervalNonlinearProblem`. This is the
    case where you're given an interval `[a,b]` and need to find where `f(u) = 0` for `u`
    inside the bounds.
 3. The "steady state problem", i.e. find the `u` such that `u' = f(u) = 0`. While related
    to (1), it's not entirely the same because there's a uniquely defined privileged root.
 4. The nonlinear least squares problem, which is an under/over-constrained nonlinear system
    which might not be satisfiable, i.e. there may be no `u` such that `f(u) = 0`, and thus
    we find the `u` which minimizes `||f(u)||` in the least squares sense.

One important distinction is that (1) and (3) require the input and output sizes to be the
same, while (4) does not.

## Problem Type 1: Solving Nonlinear Systems of Equations

A nonlinear system $$f(u) = 0$$ is specified by defining a function `f(u,p)`, where `p` are
the parameters of the system. For example, the following solves the vector
equation $$f(u) = u^2 - p$$ for a vector of equations:

In [17]:
using NonlinearSolve

f(u, p) = u .* u .-p
u0 = [1.0, 1.0]
p = 2.0
prob = NonlinearProblem(f, u0, p)
sol = solve(prob)

retcode: Success
u: 2-element Vector{Float64}:
 1.4142135623730951
 1.4142135623730951

where `u0` is the initial condition for the rootfinder. Native NonlinearSolve.jl solvers use
the given type of `u0` to determine the type used within the solver and the return. Note
that the parameters `p` can be any type, but most are an AbstractArray for automatic
differentiation.

### Investigating the Solution

To investigate the solution, one can look at the elements of the `NonlinearSolution`. The
most important value is `sol.u`: this is the `u` that satisfies `f(u) = 0`. For example:

In [3]:
u = sol.u

2-element Vector{Float64}:
 1.4142135623730951
 1.4142135623730951

In [4]:
sol.resid

2-element Vector{Float64}:
 4.440892098500626e-16
 4.440892098500626e-16

In [5]:
sol.retcode

ReturnCode.Success = 1

There are multiple return codes which can mean the solve was successful, and thus we can use
the general command `SciMLBase.successful_retcode` to check whether the solution process
exited as intended:

In [6]:
SciMLBase.successful_retcode(sol)

true

In [7]:
sol.stats

SciMLBase.NLStats
Number of function evaluations:                    7
Number of Jacobians created:                       6
Number of factorizations:                          5
Number of linear solves:                           5
Number of nonlinear solver iterations:             5

### Interacting with the Solver Options